In [1]:
import numpy as np
import pandas as pd
import requests
import json
import urllib
import tensorflow as tf
import tensorflow_datasets
from tensorflow import keras
from gensim.models.callbacks import CallbackAny2Vec
%config Completer.use_jedi =False
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
import pickle

In [32]:
picks_array=np.load('picks_and_bans/picks_array.npy')
bans_array=np.load('picks_and_bans/bans_array.npy')

In [33]:
'''Getting index of the last match having 12 bans'''
till12=0
for i in bans_array:
    if i[-1]!=-1:
        break
    till12+=1

In [19]:
'''replacing last pick of the draft by the last ban of the team having 2nd last pick'''
lastpick_mod1=np.copy(picks_array[:till12])
lastpick_mod1[:,-1]=bans_array[:till12][:,-3]

In [34]:
'''replacing the last ban of the team having 2nd last pick'''
bans_mod1=np.copy(bans_array[:till12])
bans=[]
for i in bans_mod1:
    bans.append(np.append(np.delete(i,[-3]),[-1]))
bans_mod1=np.array(bans)

In [35]:
'''doing the same for matches having 14 bans'''
lastpick_mod2=np.copy(picks_array[till12:])
lastpick_mod2[:,-1]=bans_array[till12:][:,-2]

In [36]:
lastpick_mod3=np.copy(picks_array[till12:])
lastpick_mod3[:,-1]=bans_array[till12:][:,-4]

In [37]:
bans_mod2=np.copy(bans_array[till12:])
bans=[]
for i in bans_mod2:
    bans.append(np.append(np.delete(i,[-2,-4]),[-1,-1]))

bans_mod2=np.array(bans)
bans_mod2=np.concatenate((bans_mod2,bans_mod2))

In [38]:
#concatenating the aritficially formed match data with the actualy match data 
increment_pick_array=np.concatenate((lastpick_mod1,lastpick_mod2,lastpick_mod3))
np.save('picks_and_bans\\increment_pick_array',increment_pick_array)

In [39]:
increment_bans_array=np.concatenate((bans_mod1,bans_mod2))
np.save('picks_and_bans\\increment_bans_array',increment_bans_array)

In [40]:
increment_pick_array=np.load('picks_and_bans\\increment_pick_array.npy')
modified_pick_array=np.append(picks_array,increment_pick_array,axis=0)
np.save('picks_and_bans\\extended_pick_array',modified_pick_array)
increment_bans=np.load('picks_and_bans\\increment_bans_array.npy',allow_pickle=True)
extended_bans_array=np.append(bans_array,increment_bans_array,axis=0)
np.save('picks_and_bans\\extended_bans_array',extended_bans_array)

In [ ]:
##########################################Data Structuring for Gensim Embeddings ##########################################

In [41]:
#Creating array of matches holding the names rather than the id
modified_pick_array=np.load('picks_and_bans\\extended_pick_array.npy')
hero_df=pd.read_csv('hero_df.csv')
hero_dict={}
for i in hero_df['sid']:
    hero_dict[i]=hero_df[hero_df['sid']==i]['localized_name'].values[0]
names_array=[]
for i in modified_pick_array:
    filler=[]
    for j in i:
        filler.append(hero_dict[j])
    names_array.append(filler)
modified_names_array=np.array(names_array)

In [42]:
modified_bans_array=np.load('picks_and_bans\\extended_bans_array.npy')
hero_df=pd.read_csv('hero_df.csv')
hero_dict={}
for i in hero_df['sid']:
    hero_dict[i]=hero_df[hero_df['sid']==i]['localized_name'].values[0]
bans_names_array=[]
for i in modified_bans_array:
    filler=[]
    for j in i:
        if j==-1:
            filler.append('empty')
        else:
            filler.append(hero_dict[j])
    bans_names_array.append(filler)
modified_bans_names_array=np.array(bans_names_array)

In [43]:
#shuffling the data
np.random.seed(42)
p=np.random.permutation(modified_names_array.shape[0])
print(p)
modified_pick_array=modified_pick_array[p]
modified_names_array=modified_names_array[p]
modified_names_array=modified_names_array.tolist()
np.save('picks_and_bans\\modified_names_array',modified_names_array)
np.save('picks_and_bans\\modified_pick_array',modified_pick_array)

[ 82439  20584  52366 ... 103694    860  15795]


In [44]:
#shuffling the data
np.random.seed(42)
p=np.random.permutation(modified_bans_names_array.shape[0])
modified_bans_array=modified_bans_array[p]
modified_bans_names_array=modified_bans_names_array[p]
# modified_names_array=modified_names_array.tolist()
np.save('picks_and_bans\\modified_bans_names_array',modified_bans_names_array)
np.save('picks_and_bans\\modified_bans_array',modified_bans_array)